# Data Migration - Program A

In [1]:
# Dependencies
import pandas as pd
import datetime as dt
from datetime import date
import warnings
import numpy as np
warnings.filterwarnings('ignore')

## Prepare Data Migration for Patent Family

In [ ]:
# READING DATA for Patent from Program A
prg-A_patent_df = pd.read_excel("Program A_Sample Data for Patent.xlsx", encoding = "utf8", dtype=str)
prg-A_patent_df

In [ ]:
# We want to validate values between Patent Family and Priority Application
# Patent Family Format: [Patent Holder]_[Priority Application]
# Example: 
# Patent Family: Disney_US2000-112233
# Priority Application: US2000-112233

# Extract separately patent holder and priority application in 'Patent Family' column
fam = prg-A_patent_df['Patent Family']
priority_app_fam = []

for f in fam:
    if "_" in f:
        priority_app_fam.append(f.split("_")[1])
    else:
        priority_app_fam.append("")
        
prg-A_patent_df["Priority App_from Family"] = priority_app_fam
prg-A_patent_df["Priority App_from Family"] = prg-A_patent_df["Priority App_from Family"].str.strip()

# Validate priority application we get from patent family is the same with values in 'Earliest Priority Application' column
priority_app = prg-A_patent_df['Earliest Priority Application']

prg-A_patent_df.loc[prg-A_patent_df['Priority App_from Family'] == priority_app, 'Validate_Priority App'] = 'YES'
prg-A_patent_df.loc[prg-A_patent_df['Priority App_from Family'] != priority_app, 'Validate_Priority App'] = 'NO'

diff_priority_app_index = prg-A_patent_df.index[prg-A_patent_df['Validate_Priority App'] == 'NO'].tolist()

for i in diff_priority_app_index:
    prg-A_patent_df['Earliest Priority Application'][i] = prg-A_patent_df['Priority App_from Family'][i]
    print(prg-A_patent_df.iloc[i])

In [ ]:
# Replace patent number "20050873 (Superseded)" with "200500873 (Superseded)"
# prg-A_patent_df = prg-A_patent_df.replace(["20050873 (Superseded)"], "200500873 (Superseded)")
# prg-A_patent_df = prg-A_patent_df.replace(["ZA20050873 (Superseded)"], "ZA200500873 (Superseded)")

# Extract separately patent number and "Superseded" (if any) in 'Patent Number' column
patent_no = prg-A_patent_df['Patent Number'].astype(str)
patent_no_clean = []
superseded = []

for p in patent_no:
    if "(" in p:
        patent_no_clean.append(p.split("(")[0])
        superseded.append(p.split("(")[1])
    elif "_" in p:
        patent_no_clean.append(p.split("_")[0])
        superseded.append(p.split("_")[1])
    else:
        patent_no_clean.append(p)
        superseded.append("")
        
prg-A_patent_df["Patent Number_Clean"] = patent_no_clean
prg-A_patent_df["Patent Number_Clean"] = prg-A_patent_df["Patent Number_Clean"].str.strip()
prg-A_patent_df["Superseded?"] = superseded
prg-A_patent_df["Superseded?"] = prg-A_patent_df["Superseded?"].str.replace(')','')
prg-A_patent_df["Superseded?"] = prg-A_patent_df["Superseded?"].str.strip()

# Clean the same for 'Country and Patent Number' column
# Extract separately country & patent number and "Superseded" (if any) in 'Country and Patent Number' column
CP_no = prg-A_patent_df['Country and Patent Number'].astype(str)
CP_clean = []

for cp in CP_no:
    if "(" in cp:
        CP_clean.append(cp.split("(")[0])
    elif "_" in cp:
        CP_clean.append(cp.split("_")[0])
    else:
        CP_clean.append(cp)
        
prg-A_patent_df["Country and Patent No_Clean"] = CP_clean
prg-A_patent_df["Country and Patent No_Clean"] = prg-A_patent_df["Country and Patent No_Clean"].str.strip()

# The same cleaning for 'Parent Patent Number'
# Extract separately parent patent number and "Superseded" (if any) in 'Parent Patent Number' column

PP_no = prg-A_patent_df['Parent Patent Number'].astype(str)
PP_clean = []
PP_superseded = []

for pp in PP_no:
    if "(" in pp:
        PP_clean.append(pp.split("(")[0])
        PP_superseded.append(pp.split("(")[1])
    elif "_" in pp:
        PP_clean.append(pp.split("_")[0])
        PP_superseded.append(pp.split("_")[1])
    else:
        PP_clean.append(pp)
        PP_superseded.append("")
        
prg-A_patent_df["Parent Patent No_Clean"] = PP_clean
prg-A_patent_df["Parent Patent No_Clean"] = prg-A_patent_df["Parent Patent No_Clean"].str.strip()
prg-A_patent_df["Parent Patent Superseded?"] = PP_superseded
prg-A_patent_df["Parent Patent Superseded?"] = prg-A_patent_df["Parent Patent Superseded?"].str.replace(')','')
prg-A_patent_df["Parent Patent Superseded?"] = prg-A_patent_df["Parent Patent Superseded?"].str.strip()
prg-A_patent_df

In [ ]:
# Validate each unique 'Earliest Priority Application' only has one unique 'Earliest Priority Date'
priority_check = prg-A_patent_df.pivot_table(values='Earliest Priority Date', index='Earliest Priority Application', aggfunc=pd.Series.nunique)
priority_check = priority_check.reset_index()
# priority_check
fix = list(priority_check.loc[priority_check['Earliest Priority Date'] > 1]['Earliest Priority Application'])
for app in fix:
    print(app)
    print(prg-A_patent_df.loc[prg-A_patent_df['Earliest Priority Application'] == app]['Earliest Priority Date'])

In [ ]:
# Replace and update the 'Earliest Priority Date'
for i in range(0, len(prg-A_patent_df['Earliest Priority Application'])):
    if prg-A_patent_df['Earliest Priority Application'][i] == 'KR2007-0109729' and prg-A_patent_df['Earliest Priority Date'][i] == '2009-10-30 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2007-10-30 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE1999-03552' and prg-A_patent_df['Earliest Priority Date'][i] == '1999-01-10 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '1999-10-01 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE2000-04163' and prg-A_patent_df['Earliest Priority Date'][i] == '2002-07-04 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2000-11-14 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE2000-04187' and prg-A_patent_df['Earliest Priority Date'][i] == '2001-11-15 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2000-11-15 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'US2009-213593' and prg-A_patent_df['Earliest Priority Date'][i] == '2009-12-23 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2009-06-23 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'US2009-145223' and pd.isnull(prg-A_patent_df['Earliest Priority Date'][i]) == True:
        prg-A_patent_df['Earliest Priority Date'][i] = '2009-01-16 00:00:00'

# Re-checking each unique priority application only has one unique priority date
priority_check_2 = prg-A_patent_df.pivot_table(values='Earliest Priority Date', index='Earliest Priority Application', aggfunc=pd.Series.nunique)
priority_check_2 = priority_check_2.reset_index()
check = list(priority_check_2.loc[priority_check_2['Earliest Priority Date'] > 1]['Earliest Priority Application'])
check

In [ ]:
prg-A_patent_df.loc[prg-A_patent_df['Earliest Priority Date'].isnull() == True]['Patent Family'].unique()

In [ ]:
# Checking the all family information of child patents are the same as the family information of parent patents
# Separately extract and filter parent/CEP patents and child/FC patents
CEP_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == True]
FC_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == False]

CEP_patent_filtered_df = CEP_patent_df[['Patent: Matter Number','Patent Number_Clean','Primary Patent Country',
                                       'Earliest Priority Application', 'Earliest Priority Date']]
FC_patent_filtered_df = FC_patent_df[['Patent: Matter Number','Patent Number_Clean','Primary Patent Country',
                                      'Earliest Priority Application', 'Earliest Priority Date',
                                      'Parent Patent No_Clean','Parent Patent']]
CEP_patent_filtered_df.columns = ['Matter Number_Parent','CEP_Patent Number_Clean','CEP_Primary Patent Country',
                                  'CEP_Earliest Priority Application', 'CEP_Earliest Priority Date']
FC_patent_filtered_df.columns = ['Matter Number','FC_Patent Number_Clean','FC_Primary Patent Country',
                                 'FC_Earliest Priority Application', 'FC_Earliest Priority Date',
                                 'FC_Parent Patent No_Clean','Matter Number_Parent']

# Merge/Join both dataframes
FC_PP_validation = pd.merge(FC_patent_filtered_df, CEP_patent_filtered_df, on='Matter Number_Parent', how='left')
print("Vouched Patent: {}".format(FC_patent_filtered_df.shape))
print("Certified Patent: {}".format(CEP_patent_filtered_df.shape))
print("Merged Data: {}".format(FC_PP_validation.shape))

# Validate the family information between child and parent patents
FC_priority_app = FC_PP_validation['FC_Earliest Priority Application']
FC_priority_date = FC_PP_validation['FC_Earliest Priority Date']
FC_PP = FC_PP_validation['FC_Parent Patent No_Clean']

CEP_priority_app = FC_PP_validation['CEP_Earliest Priority Application']
CEP_priority_date = FC_PP_validation['CEP_Earliest Priority Date']
CEP_patent_no = FC_PP_validation['CEP_Patent Number_Clean']

FC_PP_validation.loc[FC_PP == CEP_patent_no, 'Validate_FC-PP_Parent Patent No'] = 'YES'
FC_PP_validation.loc[FC_PP != CEP_patent_no, 'Validate_FC-PP_Parent Patent No'] = 'NO'

FC_PP_validation.loc[FC_priority_app == CEP_priority_app, 'Validate_FC-PP_Priority App'] = 'YES'
FC_PP_validation.loc[FC_priority_app != CEP_priority_app, 'Validate_FC-PP_Priority App'] = 'NO'

FC_PP_validation.loc[FC_priority_date == CEP_priority_date, 'Validate_FC-PP_Priority Date'] = 'YES'
FC_PP_validation.loc[FC_priority_date != CEP_priority_date, 'Validate_FC-PP_Priority Date'] = 'NO'

diff_PP_no = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Parent Patent No'] == 'NO']
diff_priority_app = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Priority App'] == 'NO']
diff_priority_date = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Priority Date'] == 'NO']

print(diff_PP_no)
print(diff_priority_app)
print(diff_priority_date)

In [ ]:
# Read Program A Patent List (Parent/CEP only) with PatBase Family ID
prg-A_w_Patbase_ID_raw = pd.read_excel("prg-A with PatBase Family ID_23Aug2020.xlsx", encoding = "utf8", dtype=str)
additional_data_raw = pd.read_excel("additional-data.xlsx", encoding = "utf8", dtype=str)
# These files also contains Patent Number that needs to be updated 

# Filter and rename columns
prg-A_w_Patbase_ID = prg-A_w_Patbase_ID_raw[['Patent Number','Patent Number (Need to Update)','PatBase Family ID']]
prg-A_w_Patbase_ID.columns = ['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']

additional_data_raw['Patent Number (Need to Update)'] = ''
additional_data = additional_data_raw[['Patent Number','Patent Number (Need to Update)','PatBase Family']]
additional_data.columns = ['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']

comb_w_Patbase_ID = pd.concat([prg-A_w_Patbase_ID, additional_data])

# Extract Parent/CEP and Child/FC Patents separately from prg-A_patent_df
CEP_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == True]
FC_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == False]

# Merge prg-A patent data with PatBase Family ID separately (certified merge with patent number, vouched merge with parent patent number)
# Certified
CEP_patent_df = pd.merge(CEP_patent_df, comb_w_Patbase_ID, on='Patent Number_Clean', how='left')

# Vouched
prg-A_w_Patbase_ID_as_parent = comb_w_Patbase_ID[['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']]
prg-A_w_Patbase_ID_as_parent.columns = ['Parent Patent No_Clean','Parent Patent Number (Need to Update)','PatBase Family ID']
FC_patent_df = pd.merge(FC_patent_df, prg-A_w_Patbase_ID_as_parent, on='Parent Patent No_Clean', how='left')

# Concat Certified and Vouched patent data 
prg-A_patent_concat = pd.concat([CEP_patent_df, FC_patent_df])
prg-A_patent_concat

In [ ]:
prg-A_patent_concat[prg-A_patent_concat[['Patent: Matter Number']].duplicated()==True]

In [ ]:
prg-A_patent_concat.loc[prg-A_patent_concat['PatBase Family ID'].isnull()==True]

In [ ]:
prg-A_patent_df = prg-A_patent_concat[['Patent Holder', 'Patent: Matter Number', 'Patent Number',
                                   'Publication Number', 'Primary Patent Country',
                                   'Patent Expiration Date', 'EA EP GC Counterparts', 'Patent Family',
                                   'Earliest Priority Application', 'Earliest Priority Date',
                                   'Wireless Innovation Partnership Group', 'Patent Status',
                                   'Joint Ownership Percentage', 'Country and Patent Number',
                                   'Essentiality Status', 'Parent Patent Number', 'Parent Patent',
                                   'Patent: ID', 'Priority App_from Family', 'Validate_Priority App',
                                   'Patent Number_Clean', 'Superseded?', 'Country and Patent No_Clean',
                                   'Parent Patent No_Clean', 'Parent Patent Superseded?',
                                   'PatBase Family ID', 'Patent Number (Need to Update)','Pool Exit Date']]
prg-A_patent_df = prg-A_patent_df.reset_index(drop=True)
prg-A_patent_df

In [ ]:
# Validate unique priority application extracted from certified df is the same as extracted from all prg-A patent df
priority_app_from_ALL = prg-A_patent_df['Earliest Priority Application'].unique()
print(len(priority_app_from_ALL))
priority_app_from_CEP = CEP_patent_df['Earliest Priority Application'].unique()
print(len(priority_app_from_CEP))

In [ ]:
# Double check each Earliest Priority Application is associated to one unique PatBase Family ID
patbase_fam_check = prg-A_patent_df.pivot_table(values='PatBase Family ID', index='Earliest Priority Application', aggfunc=pd.Series.nunique)
patbase_fam_check = patbase_fam_check.reset_index()

patbase_fam_fix = list(patbase_fam_check.loc[patbase_fam_check['PatBase Family ID'] > 1]['Earliest Priority Application'])
patbase_fam_fix

In [ ]:
prg-A_patent_df.columns

In [ ]:
# Filter prg-A data with Family information
prg-A_priority_info = prg-A_patent_df[['Earliest Priority Application_Updated', 'Patent Family', 'Earliest Priority Date', 'PatBase Family ID']]
prg-A_priority_info.columns = ['Earliest Priority Application', 'Patent Family', 'Earliest Priority Date', 'PatBase Family ID']
prg-A_priority_info

In [ ]:
# Filter prg-A data and extract family and unique priority date using pivot_table
prg-A_family_data = pd.pivot_table(data=prg-A_priority_info, index='Earliest Priority Application',\
                                 values=['Patent Family', 'Earliest Priority Date','PatBase Family ID'], aggfunc=pd.Series.unique)
prg-A_family_data = prg-A_family_data.reset_index()

# Rename the columns
prg-A_family_data = prg-A_family_data[['Patent Family','Earliest Priority Application','PatBase Family ID','Earliest Priority Date']]
prg-A_family_data.columns = ['Legacy LicAcc Patent Family', 'Earliest Priority Application', 'PatBase Patent Family ID', 'Priority Date']
prg-A_family_data

In [ ]:
# Ensure all earliest priority application has PatBase Family ID associated
print(prg-A_family_data.loc[prg-A_family_data['PatBase Patent Family ID'] == ""])
print(prg-A_family_data.loc[prg-A_family_data['PatBase Patent Family ID'].isnull() == True])

In [ ]:
# Re-formatting before exporting to excel
prg-A_family_data['Priority Date'] = pd.to_datetime(prg-A_family_data['Priority Date'], infer_datetime_format=True)
prg-A_family_data['Priority Date'] = prg-A_family_data['Priority Date'].dt.strftime('%Y-%m-%d')
prg-A_family_data

In [ ]:
# EXPORT DATA (Patent Family - UNIQUE)
prg-A_family_data.to_excel("Patent Project_prg-A_Patent Family_Data Migration_17Nov2020_v6.xlsx", index=False)

In [ ]:
# We also want to export NON UNIQUE Family Information
# Rearrange and Rename Columns
prg-A_priority_info = prg-A_priority_info[['Patent Family','Earliest Priority Application','PatBase Family ID','Earliest Priority Date']]
prg-A_priority_info.columns = ['Legacy LicAcc Patent Family', 'Earliest Priority Application', 'PatBase Patent Family ID', 'Priority Date']

# Re-formatting before exporting to excel
prg-A_priority_info['Priority Date'] = pd.to_datetime(prg-A_priority_info['Priority Date'], infer_datetime_format=True)
prg-A_priority_info['Priority Date'] = prg-A_priority_info['Priority Date'].dt.strftime('%Y-%m-%d')
prg-A_priority_info

# EXPORT DATA (Patent Family - Separate Rows/NON UNIQUE)
prg-A_priority_info.to_excel("Patent Project_prg-A_Patent Family-Separate Rows_Data Migration_17Nov2020_v2.xlsx", index=False)

## Prepare Data Migration for Patent 

In [ ]:
prg-A_patent_df.columns

In [ ]:
# Validate each unique patent number only has one unique expiration date
expiration_check = prg-A_patent_df.pivot_table(values='Patent Expiration Date', index='Patent Number_Clean', aggfunc=pd.Series.nunique)
expiration_check = expiration_check.reset_index()
exp_fix = list(expiration_check.loc[expiration_check['Patent Expiration Date'] > 1]['Patent Number_Clean'])
exp_fix

In [ ]:
# Get the expiration date of currently active/expired patents
patent = prg-A_patent_df['Patent Number_Clean']
exp_date = prg-A_patent_df['Patent Expiration Date']
patent_status = prg-A_patent_df['Patent Status'].astype(str)
updated_exp_date = []
patent_list = []

for x in range(0, len(patent)):
    if (prg-A_patent_df.iat[x,11] == 'Effective (Active)') or (prg-A_patent_df.iat[x,11] == 'Expired'):
        updated_exp_date.append(prg-A_patent_df.iat[x,5])
        patent_list.append(prg-A_patent_df.iat[x,20])
    elif (prg-A_patent_df.iat[x,11] == 'Pending for Approval') and (prg-A_patent_df.iat[x,20] not in patent_list):
        updated_exp_date.append(prg-A_patent_df.iat[x,5])
        patent_list.append(prg-A_patent_df.iat[x,20])
    elif (prg-A_patent_df.iat[x,11] == 'Superseded') and (prg-A_patent_df.iat[x,27] == '2020-09-30 00:00:00'):
        updated_exp_date.append(prg-A_patent_df.iat[x,5])
        patent_list.append(prg-A_patent_df.iat[x,20])

patent_exp_date_df = pd.DataFrame({'Patent Number': patent_list, 'Updated Expiration Date': updated_exp_date})
patent_exp_date_df

In [ ]:
# Checking the same counts of unique patent number for original patent_df and patent_exp_date_df
# len(prg-A_patent_df['Patent Number_Clean'].unique())
# len(patent_exp_date_df["Patent Number"].unique())

# Check if all patent number in original patent_df exists in patent_exp_date_df
for pn in prg-A_patent_df['Patent Number_Clean'].unique():
    if pn not in patent_exp_date_df["Patent Number"].unique():
        print(pn)

In [ ]:
# Same as original patent_df, check if each unique patent number in patent_exp_date_df, 
# also only has one unique expiration date
expiration_check = patent_exp_date_df.pivot_table(values='Updated Expiration Date', index='Patent Number', aggfunc=pd.Series.nunique)
expiration_check = expiration_check.reset_index()

exp_fix = list(expiration_check.loc[expiration_check['Updated Expiration Date'] > 1]['Patent Number'])
exp_fix

In [ ]:
# Extract unique patent number and expiration date for each patent using pivot_table
correct_exp_date = pd.pivot_table(data=patent_exp_date_df, index='Patent Number',\
                                 values='Updated Expiration Date', aggfunc=pd.Series.unique)
correct_exp_date = correct_exp_date.reset_index()

# Rename the columns
correct_exp_date.columns = ['Patent Number_Clean', 'Correct Expiration Date']
correct_exp_date

In [ ]:
# Now merge original patent_df with the correct expiration date for each patents, so 
# each unique patent number only has one unique expiration date
prg-A_patent_df = pd.merge(prg-A_patent_df, correct_exp_date, on="Patent Number_Clean", how='left')
prg-A_patent_df

In [ ]:
# 10. Preparing Data for Patent Object Data
# Replace comma in 'EA EP GC Counterparts' with semicolon and remove parenthesis 
prg-A_patent_df["EA EP GC Counterparts"] = prg-A_patent_df["EA EP GC Counterparts"].str.replace('(','')
prg-A_patent_df["EA EP GC Counterparts"] = prg-A_patent_df["EA EP GC Counterparts"].str.replace(')','')
prg-A_patent_df["EA EP GC Counterparts"] = prg-A_patent_df["EA EP GC Counterparts"].str.replace(',',';')
prg-A_patent_df['Correct Expiration Date'] = pd.to_datetime(prg-A_patent_df['Correct Expiration Date'], infer_datetime_format=True)

# Remove comma in 'AT&T Intellectual Property, LLC'
prg-A_patent_df["Joint Ownership Percentage"] = prg-A_patent_df["Joint Ownership Percentage"].str.replace('AT&T Intellectual Property, LLC','AT&T Intellectual Property LLC')

# Create another column for updated patent status values according to expiration date (Values: Issued or Expired or Application)
exp_date = prg-A_patent_df['Correct Expiration Date']
today_date = pd.datetime.now().date()
patent_status_updated = []

for d in exp_date:
    if today_date < d:
        patent_status_updated.append("Issued")
    elif today_date >= d:
        patent_status_updated.append("Expired")

# Remove the last comma at the end of joint ownership percentage value
# Example: 'Dolby International AB - 50.0%,Fraunhofer - 50.0%,'
ownership_percentage = []
for value in prg-A_patent_df["Joint Ownership Percentage"]:
    if (value.rfind(",")) == (len(value)-1):
        ownership_percentage.append(value[:value.rfind(",")])
    else:
        ownership_percentage.append(value)        

prg-A_patent_df["Patent Status_Updated"] = patent_status_updated
prg-A_patent_df["Application Number"] = ""
prg-A_patent_df["Last Patent Annuity Payment"] = ""
prg-A_patent_df["Last Patent Annuity Amount"] = ""
prg-A_patent_df["Note"] = ""
prg-A_patent_df["Joint Ownership Percentage_Updated"] = ownership_percentage
prg-A_patent_df.head()

In [ ]:
print(len(prg-A_patent_df.loc[prg-A_patent_df["Patent Status"] == "Expired"]))
print(len(prg-A_patent_df.loc[prg-A_patent_df["Patent Status_Updated"] == "Expired"]))
print(prg-A_patent_df["Superseded?"].unique())

In [ ]:
prg-A_patent_df.loc[prg-A_patent_df["Patent Number_Updated"].isnull() == True]

In [ ]:
prg-A_patent_df.columns

In [ ]:
# If patent number needs to be updated, we need to combine 'Patent Number_Clean' and 'Patent Number (Need to Update)'
patent_no_updated = []
country_patent_updated = []

for p in range(0,len(AAC_patent_df['Patent Number_Clean'])):
    if (pd.isnull(AAC_patent_df['Patent Number (Need to Update)'][p]) == False) and (AAC_patent_df['Patent Number (Need to Update)'][p] != ''):
        patent_no_updated.append(AAC_patent_df['Patent Number (Need to Update)'][p])
        country_patent_updated.append(AAC_patent_df['Primary Patent Country'][p]+AAC_patent_df['Patent Number (Need to Update)'][p])
    else:
        patent_no_updated.append(AAC_patent_df['Patent Number_Clean'][p])
        country_patent_updated.append(AAC_patent_df['Country and Patent No_Clean'][p])
AAC_patent_df["Patent Number_Updated"] = patent_no_updated
AAC_patent_df["Country and Patent No_Updated"] = country_patent_updated
AAC_patent_df

In [ ]:
# Extract unique patent number and unique country with all values from necessary columns for Patent Objects
prg-A_patent_data = pd.pivot_table(data=prg-A_patent_df, index=['Patent Number_Updated','Primary Patent Country'],\
                                 values=['Application Number', 'Publication Number', 'Correct Expiration Date',
                                         'EA EP GC Counterparts', 'Earliest Priority Application_Updated', 
                                         'Wireless Innovation Partnership Group', 'Last Patent Annuity Payment',
                                         'Last Patent Annuity Amount', 'Patent Status_Updated', 'Note'],\
                                 aggfunc=pd.Series.unique)
prg-A_patent_data = prg-A_patent_data.reset_index()
prg-A_patent_data

In [ ]:
# Re-arranging the columns
prg-A_patent_data["Wireless Innovation Partnership Group"] = ""
prg-A_patent_data["Patent Number (Licensor Format)"] = ""
prg-A_patent_data = prg-A_patent_data[['Patent Number_Updated', 'Application Number', 'Publication Number',
                                   'Patent Number (Licensor Format)', 'Primary Patent Country', 
                                   'Correct Expiration Date', 'EA EP GC Counterparts',
                                   'Earliest Priority Application_Updated', 'Wireless Innovation Partnership Group',
                                   'Last Patent Annuity Payment', 'Last Patent Annuity Amount',  
                                   'Patent Status_Updated', 'Note']]
# Re-naming the columns
prg-A_patent_data.columns = ['Patent Number', 'Application Number', 'Publication Number',
                           'Patent Number (Licensor Format)', 'Jurisdiction', 
                           'Expiration Date', 'Validated Designated Country',
                           'Patent Family Record', 'Wireless Innovation Partnership Group', 
                           'Last Patent Annuity Payment', 'Last Patent Annuity Amount', 
                           'Patent Status', 'Note']
prg-A_patent_data

In [ ]:
# Re-formatting before exporting
prg-A_patent_data = prg-A_patent_data.fillna("")
prg-A_patent_data['Expiration Date'] = pd.to_datetime(prg-A_patent_data['Expiration Date'], infer_datetime_format=True)
prg-A_patent_data['Expiration Date'] = prg-A_patent_data['Expiration Date'].dt.strftime('%Y-%m-%d')
prg-A_patent_data

# EXPORT DATA
prg-A_patent_data.to_excel("Patent Project_prg-A_Patent_Data Migration_17Nov2020_v5.xlsx", index=False)

## Prepare Data Migration for Licensor Ownership Object

In [ ]:
prg-A_patent_df.columns

In [ ]:
prg-A_patent_df['Patent Holder'].unique()

In [ ]:
# READING DATA
licensor_opp = pd.read_excel('.xlsx', encoding = "utf8", dtype=str)
prg-A_licensor_opp = licensor_opp.loc[licensor_opp['Program']=='prg-A']
prg-A_licensor_opp = prg-A_licensor_opp.loc[prg-A_licensor_opp['Opportunity Name']!='prg-A Sony Licensor (Old)']
prg-A_licensor_opp = prg-A_licensor_opp.reset_index(drop=True)
prg-A_licensor_opp = prg-A_licensor_opp[['Account Name', 'Opportunity Name', 'Licensor Opportunity ID']]
prg-A_licensor_opp.columns = ['Patent Holder', 'Licensor Opportunity', 'Licensor Opportunity ID']
prg-A_licensor_opp

In [ ]:
# Check if each Patent Holder only have one unique Licensor Opportunity
prg-A_licensor_opp_check = prg-A_licensor_opp.pivot_table(values='Licensor Opportunity', index='Patent Holder', aggfunc=pd.Series.nunique)
prg-A_licensor_opp_check = prg-A_licensor_opp_check.reset_index()
check = list(prg-A_licensor_opp_check.loc[prg-A_licensor_opp_check['Licensor Opportunity'] > 1]['Patent Holder'])
check

In [ ]:
# Extract unique Licensor Opportunity by each Patent Holder
prg-A_licensor_opp = pd.pivot_table(data=prg-A_licensor_opp, index='Patent Holder',\
                                  values=['Licensor Opportunity', 'Licensor Opportunity ID'], aggfunc=pd.Series.unique)
prg-A_licensor_opp = prg-A_licensor_opp.reset_index()
prg-A_licensor_opp

In [ ]:
# Prepare data for Licensor Ownership Object
# Filter dataframe for licensor ownership data
prg-A_patent_filtered = prg-A_patent_df[['Patent: Matter Number', 'Patent Holder', 'Joint Ownership Percentage_Updated']]

# There are some lines that have multiple ownership percentage in one line separated by comma
# so we will separate them into different rows
# prg-A_patent_ownership = pd.DataFrame(prg-A_patent_filtered['Joint Ownership Percentage_Updated'].str.split(',').tolist(), index=prg-A_patent_filtered['Patent: Matter Number']).stack()
# prg-A_patent_ownership = prg-A_patent_ownership.reset_index([0, 'Patent: Matter Number'])
# prg-A_patent_ownership.columns = ['Patent: Matter Number', 'Joint Ownership Percentage']
prg-A_patent_ownership = prg-A_patent_filtered.assign(separate_percentage=prg-A_patent_filtered['Joint Ownership Percentage_Updated'].str.split(',')).explode('separate_percentage')

# Extract the percetage only and put it in new column
prg-A_patent_ownership["Licensor"] = prg-A_patent_ownership['separate_percentage'].str.split(' - ').str[0]
prg-A_patent_ownership["Ownership"] = prg-A_patent_ownership['separate_percentage'].str.split(' - ').str[1]
prg-A_patent_ownership["Ownership"] = prg-A_patent_ownership["Ownership"].str.replace("%", "")
prg-A_patent_ownership["Ownership"] = prg-A_patent_ownership["Ownership"].str.replace("33.0", "33.33")
prg-A_patent_ownership["Ownership"] = prg-A_patent_ownership["Ownership"].str.replace("34.0", "33.34")
prg-A_patent_ownership["Ownership"] = prg-A_patent_ownership["Ownership"].str.replace("50.0", "50")

# # Filter and extract the columns we need and re-name the columns
prg-A_patent_ownership = prg-A_patent_ownership[['Patent: Matter Number', 'Licensor', 'Ownership']]
prg-A_patent_ownership

In [ ]:
prg-A_patent_ownership["Ownership"].unique()

In [ ]:
# Read the Account Name Mapping file
account_mapping = pd.read_excel("Account Name Mapping.xlsx", encoding = "utf8", dtype=str)
account_mapping_filter = account_mapping[['Account Name in LicAcc','Account Name in SFDC']]
account_mapping_filter['Account Name in LicAcc'] = account_mapping_filter['Account Name in LicAcc'].replace('AT&T Intellectual Property, LLC', 'AT&T Intellectual Property LLC')
account_mapping_filter.columns = ['Licensor','Account Name in SFDC']

# Merge Account Name Mapping to prg-A_patent_ownership
prg-A_patent_ownership_updated = pd.merge(prg-A_patent_ownership, account_mapping_filter, on='Licensor', how='left')
prg-A_patent_ownership_updated


In [ ]:
prg-A_patent_ownership_updated[prg-A_patent_ownership_updated['Account Name in SFDC'].isnull()==True]

In [ ]:
# Prepare data for Licensor Ownership Object
# Filter dataframe and rename for licensor ownership data
prg-A_patent_ownership_updated = prg-A_patent_ownership_updated[['Patent: Matter Number', 'Account Name in SFDC', 'Ownership']]
prg-A_patent_ownership_updated.columns = ['Related Patent In Program Record', 'Patent Holder', 'Ownership']

# Merge Licensor Opportunity data based on Patent Holder
prg-A_patent_ownership_updated = pd.merge(prg-A_patent_ownership_updated, prg-A_licensor_opp, on='Patent Holder', how='left')
prg-A_patent_ownership_updated

In [ ]:
print(prg-A_patent_ownership_updated[prg-A_patent_ownership_updated['Licensor Opportunity']==''])
print(prg-A_patent_ownership_updated[prg-A_patent_ownership_updated['Licensor Opportunity'].isnull()==True])

In [ ]:
# Filter and extract the columns we need and re-name the columns
prg-A_patent_ownership_updated = prg-A_patent_ownership_updated[['Related Patent In Program Record', 'Licensor Opportunity', 'Patent Holder', 'Ownership']]
prg-A_patent_ownership_updated.columns = ['Related Patent In Program Record', 'Licensor', 'Account', 'Ownership']
prg-A_patent_ownership_updated

In [ ]:
# EXPORT DATA
prg-A_patent_ownership_updated.to_excel("Patent Project_prg-A_Licensor Ownership_Data Migration_17Nov2020_v6.xlsx", index=False)

## Prepare Data Migration for Patent In Program Object

In [ ]:
# READING DATA
prg-A_PIP_raw = pd.read_excel("prg-A_Data for PIP_17Nov2020_v4.xlsx", encoding = "utf8", dtype=str)
prg-A_PIP_raw

In [ ]:
prg-A_PIP_raw.loc[(prg-A_PIP_raw["Parent Patent Number"].isnull()==True)&(prg-A_PIP_raw["Evaluated or Vouched"]=="Vouched")]

In [ ]:
prg-A_patent_df['Patent Status'].unique()
prg-A_patent_df['Superseded?'].unique()

In [ ]:
prg-A_patent_df.columns

In [ ]:
# Filter the columns of prg-A PIP data and prg-A Patent data
prg-A_PIP_df = prg-A_PIP_raw[['Program', 'Evaluated or Vouched', 'Patent Type',
                          'Invention Group', 'Distinct Invention Result Date',
                          'Patent Received Date', 'Vouch Notice Date', 'Pool Entry Date',
                          'Pool Exit Date', 'Disposal Date', 'Proud Patent Start Date',
                          'Proud Patent End Date', 'EPPRS X3 Point Start Date',
                          'EPPRS X3 Point End Date', 'Essential Patent Notification Date',
                          'Technologies', 'Section/Tools/Profile Details', 'Tools', 'Objects',
                          'Essential Claim', 'Standard Sections', 'Essential Claim Type',
                          'Evaluation Note', 'Notes', 'Patent: ID', 'Previous Matter Number']]

prg-A_patent_filter_PIP = prg-A_patent_df[['Patent: Matter Number', 'Country and Patent No_Clean', 'Parent Patent No_Clean',
                                       'Parent Patent', 'Patent Number_Clean', 
                                       'Patent Number_Updated', 'Country and Patent No_Updated',
                                       'Primary Patent Country', 'Patent: ID', 'Superseded?', 
                                       'Patent Status', 'Patent Family', 'Earliest Priority Application_Updated']]

In [ ]:
# Merge patent data and PIP patent data 
prg-A_PIP_df = pd.merge(prg-A_PIP_df, prg-A_patent_filter_PIP, on="Patent: ID", how='inner')
prg-A_PIP_df

In [ ]:
# Prepare data for Patent In Program Data
# Ensure patents, that have Vouch Notice Date, are the only ones with "Vouched" patent type
# prg-A_PIP_df.loc[(prg-A_PIP_df["Vouch Notice Date"].isnull() == False) & (prg-A_PIP_df['Parent Patent No_Clean'].isnull() == True)]
# prg-A_PIP_df.loc[(prg-A_PIP_df["Vouch Notice Date"].isnull() == False) & (prg-A_PIP_df['Evaluated or Vouched'] != "Vouched")]
# prg-A_PIP_df.loc[(prg-A_PIP_df["Parent Patent No_Clean"] != "nan") & (prg-A_PIP_df['Evaluated or Vouched'] != "Vouched")]

# Map the "Evaluated or Vouched" to new Patent Type column
eval_or_vouch = prg-A_PIP_df['Evaluated or Vouched']
patent_type_updated = []

for t in eval_or_vouch:
    if t == "Evaluated":
        patent_type_updated.append("Certified")
    elif t == "Vouched":
        patent_type_updated.append("Vouched")
        
received_date = prg-A_PIP_df['Patent Received Date']
vouch_received_date = []

for i in range(0, len(eval_or_vouch)):
    if eval_or_vouch[i] == "Vouched":
        vouch_received_date.append(received_date[i])
    else:
        vouch_received_date.append("")

prg-A_PIP_df["Technologies"] = prg-A_PIP_df["Technologies"].str.replace(',',';')    

prg-A_PIP_df["Patent Type_Updated"] = patent_type_updated
prg-A_PIP_df["Vouch Received Date"] = vouch_received_date
prg-A_PIP_df.head()

In [ ]:
# There are some Pool Entry Date that has a date from middle of quarter
# Create function to get the start date of following quarter from a date
def getnextQuarterStart(theDate=dt.date.today()):
    m = ((round((theDate.month - 1) // 3) + 1) * 3) + 1
    if m > 12:
        return dt.date(theDate.year + 1, 1, 1)
    else:
        return dt.date(theDate.year, m, 1)

In [ ]:
# Apply the function to Pool Entry Data Column so Pool Entry Date is always first day of quarter
prg-A_PIP_df["Pool Entry Date"] = pd.to_datetime(prg-A_PIP_df['Pool Entry Date'])

entry_date = prg-A_PIP_df["Pool Entry Date"]
entry_date_updated = []

for d in entry_date:
    if (d.date().month in [1,4,7,10]) and (d.date().day == 1):
        entry_date_updated.append(d.date())
    else:
        entry_date_updated.append(getnextQuarterStart(d.date()))
prg-A_PIP_df["Pool Entry Date_Updated"] = entry_date_updated
prg-A_PIP_df

In [ ]:
# Exclude patent that has Pool Entry Date > Pool Exit Date
prg-A_PIP_df = prg-A_PIP_df.loc[prg-A_PIP_df['Pool Entry Date_Updated'] < prg-A_PIP_df["Pool Exit Date"]]
prg-A_PIP_df = prg-A_PIP_df.reset_index(drop=True)
prg-A_PIP_df

In [ ]:
# look for the min of 'Pool Entry Date' for each patent number
# Filter prg-A data
prg-A_data_filtered = prg-A_PIP_df[["Country and Patent No_Updated", "Pool Entry Date_Updated"]]

# Convert 'Pool Entry Date' to datetime
prg-A_data_filtered["Pool Entry Date_Updated"] = pd.to_datetime(prg-A_data_filtered["Pool Entry Date_Updated"])

# Extract min 'Pool Entry Date' by using groupby
min_entry_date = prg-A_data_filtered.groupby(["Country and Patent No_Updated"], as_index=False).min()
min_entry_date.columns = ["Country and Patent No_Updated", "Earliest Pool Entry Date"]
min_entry_date

In [ ]:
# Add Earliest Pool Entry Date to prg-A_PIP_df
prg-A_PIP_df = prg-A_PIP_df.merge(min_entry_date, on='Country and Patent No_Updated', how='left')
prg-A_PIP_df

In [ ]:
prg-A_PIP_df[prg-A_PIP_df['Earliest Pool Entry Date'].isnull()==True]

In [ ]:
# Create sequence number to assign the version of each matters based on Country and Patent No_Clean and Pool Entry Date (from smallest to largest)
# First, sort data.
prg-A_PIP_df["Pool Entry Date_Updated"] = pd.to_datetime(prg-A_PIP_df["Pool Entry Date_Updated"])
prg-A_PIP_df["Earliest Pool Entry Date"] = pd.to_datetime(prg-A_PIP_df["Earliest Pool Entry Date"])
prg-A_PIP_df = prg-A_PIP_df.sort_values(['Country and Patent No_Updated', 'Pool Entry Date_Updated'], ascending=[True, True])

# Second, use cumcount() to number each item in each group from 0 to the length of that group -1. 
# Sequence number will be created in 'Version No.' column
g = prg-A_PIP_df.groupby('Country and Patent No_Updated')
prg-A_PIP_df['Version No.'] = g.cumcount() + 1
prg-A_PIP_df

In [ ]:
prg-A_PIP_df['Version No.'].unique()

In [ ]:
# Reset Index
prg-A_PIP_df = prg-A_PIP_df.reset_index(drop=True)
prg-A_PIP_df

In [ ]:
# Find patent evaluation matters in prg-A program from prg-B, prg-C, prg-A evaluations
# Read prg-B, prg-C, prg-A patent evaluation excel data
prg-A_eval = pd.read_excel("prg-A_Data for Evaluation.xlsx", encoding = "utf8", dtype=str)
prg-B_eval = pd.read_excel("prg-B_Data for Evaluation.xlsx", encoding = "utf8", dtype=str)
prg-C_eval = pd.read_excel("prg-C_Data for Evaluation.xlsx", encoding = "utf8", dtype=str)

# Filter MPEG-4 and prg-C patent evaluations data
prg-B_eval = prg-B_eval.loc[(prg-B_eval["Patent Status"] == "Effective (Active)") | \
                        (prg-B_eval["Patent Status"] == "Expired") |\
                        (prg-B_eval["Patent Status"] == "Superseded")]
prg-C_eval = prg-C_eval.loc[prg-C_eval["Patent Status"] == "Effective (Active)"]

print(len(prg-B_eval))
print(len(prg-C_eval))

# prg-A Data Cleaning
# Extract separately patent number and "Superseded" (if any) in 'Patent Number' column
CP_no = prg-A_eval['Country and Patent Number'].astype(str)
CP_clean = []

for cp in CP_no:
    if "(" in cp:
        CP_clean.append(cp.split("(")[0])
    else:
        CP_clean.append(cp)
        
prg-A_eval["Country and Patent No_Clean"] = CP_clean
prg-A_eval["Country and Patent No_Clean"] = prg-A_eval["Country and Patent No_Clean"].str.strip()
prg-A_eval

In [ ]:
# prg-B Data Cleaning
# Extract separately patent number and "Superseded" (if any) in 'Patent Number' column
patent_no = prg-B_eval['Patent Number'].astype(str)
patent_no_clean = []
superseded = []

for p in patent_no:
    if "(" in p:
        patent_no_clean.append(p.split("(")[0])
        superseded.append(p.split("(")[1])
    elif "_" in p:
        patent_no_clean.append(p.split("_")[0])
        superseded.append(p.split("_")[1])
    else:
        patent_no_clean.append(p)
        superseded.append("")
        
prg-B_eval["Patent Number_Clean"] = patent_no_clean
prg-B_eval["Patent Number_Clean"] = prg-B_eval["Patent Number_Clean"].str.strip()
prg-B_eval["Superseded?"] = superseded
prg-B_eval["Superseded?"] = prg-B_eval["Superseded?"].str.replace(')','')
prg-B_eval["Superseded?"] = prg-B_eval["Superseded?"].str.strip()

# Clean the same for Country and Patent Number
CP_no = prg-B_eval['Country and Patent Number'].astype(str)
CP_clean = []

for cp in CP_no:
    if "(" in cp:
        CP_clean.append(cp.split("(")[0])
    elif "_" in cp:
        CP_clean.append(cp.split("_")[0])
    else:
        CP_clean.append(cp)
        
prg-B_eval["Country and Patent No_Clean"] = CP_clean
prg-B_eval["Country and Patent No_Clean"] = prg-B_eval["Country and Patent No_Clean"].str.strip()

# prg-C Data Cleaning
# Extract separately patent number and "Superseded" (if any) in 'Patent Number' column
patent_no = prg-C_eval['Patent Number'].astype(str)
patent_no_clean = []
superseded = []

for p in patent_no:
    if "(" in p:
        patent_no_clean.append(p.split("(")[0])
        superseded.append(p.split("(")[1])
    elif "_" in p:
        patent_no_clean.append(p.split("_")[0])
        superseded.append(p.split("_")[1])
    else:
        patent_no_clean.append(p)
        superseded.append("")
        
prg-C_eval["Patent Number_Clean"] = patent_no_clean
prg-C_eval["Patent Number_Clean"] = prg-C_eval["Patent Number_Clean"].str.strip()
prg-C_eval["Superseded?"] = superseded
prg-C_eval["Superseded?"] = prg-C_eval["Superseded?"].str.replace(')','')
prg-C_eval["Superseded?"] = prg-C_eval["Superseded?"].str.strip()

# Clean the same for Country and Patent Number
CP_no = prg-C_eval['Country and Patent Number'].astype(str)
CP_clean = []

for cp in CP_no:
    if "(" in cp:
        CP_clean.append(cp.split("(")[0])
    if "_" in cp:
        CP_clean.append(cp.split("_")[0])
    else:
        CP_clean.append(cp)
        
prg-C_eval["Country and Patent No_Clean"] = CP_clean
prg-C_eval["Country and Patent No_Clean"] = prg-C_eval["Country and Patent No_Clean"].str.strip()

print(len(prg-B_eval["Country and Patent No_Clean"].unique()))
print(len(prg-C_eval["Country and Patent No_Clean"].unique()))